In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
X_train = pd.read_csv('/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/DATA/PROCESSED/X_train_processed.csv')
X_test = pd.read_csv('/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/DATA/PROCESSED/X_test_processed.csv')
y_train = pd.read_csv('/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/DATA/PROCESSED/y_train_cat.csv')
y_test = pd.read_csv('/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/DATA/PROCESSED/y_test_cat.csv')

X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.2,random_state = 40)

In [3]:
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
X_val = X_val.values.reshape(-1,28,28,1)

In [8]:
model = Sequential([
    Conv2D(64,(3,3),activation = 'relu', input_shape = (28,28,1),padding = 'same'), BatchNormalization(),
    Conv2D(64,(3,3),activation = 'relu',padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128,(3,3),activation = 'relu',padding = 'same'),
    BatchNormalization(),
    Conv2D(128,(3,3),activation = 'relu',padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Flatten(),
    Dense(256,activation = 'relu'),
    Dropout(0.5),
    Dense(10,activation = 'softmax')
])

/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model_path = "/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/MODELS/best_model.keras"
early_stop = EarlyStopping(monitor= 'val_loss', patience = 15, restore_best_weights = True)
lr_reduce = ReduceLROnPlateau(monitor = 'val_loss', patience = 8, factor = 0.5, verbose = 1)
checkpoint = ModelCheckpoint(model_path,monitor = 'val_accuracy',save_best_only = True, verbose = 1)
datagen = ImageDataGenerator(rotation_range = 10,zoom_range = 0.1, horizontal_flip = True)
datagen.fit(X_train)

In [10]:
model.compile(optimizer = Adam(learning_rate = 0.001),loss = 'categorical_crossentropy',metrics = ['accuracy'])


history = model.fit(datagen.flow(X_train,y_train,batch_size = 64), validation_data = (X_val,y_val), epochs = 100, callbacks = [early_stop,lr_reduce,checkpoint])

Epoch 1/100


/Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.5916 - loss: 1.2967
Epoch 1: val_accuracy improved from -inf to 0.80508, saving model to /Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/MODELS/best_model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 155s 206ms/step - accuracy: 0.5918 - loss: 1.2962 - val_accuracy: 0.8051 - val_loss: 0.5514 - learning_rate: 0.0010
Epoch 2/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.7698 - loss: 0.6169
Epoch 2: val_accuracy improved from 0.80508 to 0.85808, saving model to /Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/MODELS/best_model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 152s 203ms/step - accuracy: 0.7698 - loss: 0.6169 - val_accuracy: 0.8581 - val_loss: 0.3944 - learning_rate: 0.0010
Epoch 3/100
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.8166 - loss: 0.5093
Epoch 3: val_accuracy improved from 0.85808 to 0.88000, saving model to /Users/mr.engineer/Desktop/Code/DSPROJECT/MNIST_FASHION/MODELS/best_model.keras
75

In [11]:
train_loss, train_accuracy = model.evaluate(X_train,y_train)
print("Test Accuracy: ",train_accuracy)

val_loss, val_accuracy = model.evaluate(X_val,y_val)
print("Test Accuracy: ",val_accuracy)

test_loss, test_accuracy = model.evaluate(X_test,y_test)
print("Test Accuracy: ",test_accuracy)

1500/1500 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.9716 - loss: 0.0795
Test Accuracy:  0.9708333611488342
375/375 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9395 - loss: 0.1830
Test Accuracy:  0.9410833120346069
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.9466 - loss: 0.1645
Test Accuracy:  0.9462000131607056
